In [1]:
# Import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [4]:
# Create a variable with the URL to this tutorial
url = 'http://www.englishforum.ch/forum.php'
# Scrape the HTML at the url
r = requests.get(url)
# Turn the HTML into a Beautiful Soup object
soup = BeautifulSoup(r.text, 'lxml')

In [21]:
# Create four variables to score the scraped data in
dataframes={}
for link in soup.find_all("strong")[1:]:
    for linkhref in soup.find_all("a"):
        if(link in linkhref and not linkhref['href'].endswith('.html') and link.contents[0] not in dataframes):
            dataframes[link.contents[0]]=linkhref['href']
dataframes


{'Business & entrepreneur': 'http://www.englishforum.ch/business-entrepreneur/',
 'Commercial events': 'http://www.englishforum.ch/commercial-events/',
 'Complaints corner': 'http://www.englishforum.ch/complaints-corner/',
 'Concerts': 'http://www.englishforum.ch/concerts/',
 'Daily life': 'http://www.englishforum.ch/daily-life/',
 'Education': 'http://www.englishforum.ch/education/',
 'Employment': 'http://www.englishforum.ch/employment/',
 'Entertainment & dining': 'http://www.englishforum.ch/entertainment-dining/',
 'Family matters/health': 'http://www.englishforum.ch/family-matters-health/',
 'Finance/banking/taxation': 'http://www.englishforum.ch/finance-banking-taxation/',
 'Food and drink': 'http://www.englishforum.ch/food-drink/',
 'Housing in general': 'http://www.englishforum.ch/housing-general/',
 'Insurance': 'http://www.englishforum.ch/insurance/',
 'Introductions': 'http://www.englishforum.ch/introductions/',
 'Language corner': 'http://www.englishforum.ch/language-corner

# Fetch the info from urls

In [6]:
from bs4.element import Tag

def page(url,index):
#     print(url,index)
    # Scrape the HTML at the url
    r = requests.get(url)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    subtopic={}
    for link in soup.find_all("table",id="threadslist"):
    #     print(link)
        index_aux=index
        dash=0
        for replies in link.find_all("td",class_="alt1"):
            if(replies.contents[0]!=' '):
                if(replies.contents[0]=='-'):
                    dash+=1
                subtopic[index_aux]=[]
                subtopic[index_aux].append(replies.contents[0])
# #                 print('rep',index_aux,replies.contents[0])

                index_aux+=1
        index_aux=index
        insertdash=0
        for views in link.find_all("td",class_="alt2"):
            if(views.contents[0]!=' '):
                if(views.contents[0]=='-'):
                    if(insertdash==0):
                        insertdash=1
                    else:
                        insertdash=0
#                         print('views',index_aux,views.contents[0])
                        subtopic[index_aux].append(views.contents[0])
                        index_aux+=1
                else:
#                     print('views',index_aux,views.contents[0])
#                 if(index_aux not in subtopic):
#                     print('errorrrrrrr',index_aux,views.contents[0])
#                     next
                    subtopic[index_aux].append(views.contents[0])
                    index_aux+=1        
#         index_aux=index
        for thread in link.find_all("a"):
            if('id' in thread.attrs):
#                 print('hrefs',index,link1.contents[0])
                subtopic[index].append(thread['href'])
                subtopic[index].append(thread.contents[0])
                index+=1
    return [subtopic,index]
    
    
def findAllPages(rootHTML):
    # Scrape the HTML at the url
    r = requests.get(rootHTML)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    lastUrl=''
    pages=[]
    pages.append(rootHTML)
    for link in soup.find_all("body"):
        for link1 in link.find_all("a"):
            if('title' in link1.attrs and 'Last' in link1['title']):
                lastUrl=link1['href']
                break
    if(lastUrl!=''):
        limit=re.findall(r'\d+',lastUrl)[0]
        for i in range(2,int(limit)):
            pages.append(rootHTML+"index"+str(i)+".html")
        pages.append(lastUrl)
        return pages
    return pages
    
def createDF(url):
#     print(url)
    index=0
    allsubtopics={}
    nextPages=findAllPages(url)
    for url in nextPages:
#         print('urlllll',url)
        result=page(url,index)
        allsubtopics.update(result[0])
        index=result[1]
    #         print('fin',subtopic)
    replies=[]
    views=[]
    hrefs=[]
    threads=[]
    for value in allsubtopics.values():
        replies.append(value[0])
        views.append(value[1])
        hrefs.append(value[2])
        threads.append(value[3])
    df = pd.DataFrame({'replies': replies, 'views': views, 'hrefs': hrefs, 'threads' : threads})
    return df

# Save the data into csv folder and remove dataframe

In [37]:
def saveData(dataframe):
    dataframe.to_csv("csv/"+i.replace("/","")+".csv", sep='\t', encoding='utf-8',)
    del dataframe

# Dataframes we'd like to fetch

In [23]:
# for i in dataframes:
for index,i in enumerate(sorted(dataframes)):
    print(index,i,dataframes[i])

0 Business & entrepreneur http://www.englishforum.ch/business-entrepreneur/
1 Commercial events http://www.englishforum.ch/commercial-events/
2 Complaints corner http://www.englishforum.ch/complaints-corner/
3 Concerts http://www.englishforum.ch/concerts/
4 Daily life http://www.englishforum.ch/daily-life/
5 Education http://www.englishforum.ch/education/
6 Employment http://www.englishforum.ch/employment/
7 Entertainment & dining http://www.englishforum.ch/entertainment-dining/
8 Family matters/health http://www.englishforum.ch/family-matters-health/
9 Finance/banking/taxation http://www.englishforum.ch/finance-banking-taxation/
10 Food and drink http://www.englishforum.ch/food-drink/
11 Housing in general http://www.englishforum.ch/housing-general/
12 Insurance http://www.englishforum.ch/insurance/
13 Introductions http://www.englishforum.ch/introductions/
14 Language corner http://www.englishforum.ch/language-corner/
15 Leaving Switzerland http://www.englishforum.ch/leaving-switzerl

# In order to read from csv files

In [35]:
def readData(filename):
    if( not filename.endswith('.csv')):
        filename+='.csv'
    df = pd.read_csv('csv/'+filename, header=0,sep='\t',index_col=0)
    return df

# CSV files we have got so far

In [24]:
for index,i in enumerate(sorted(os.listdir("csv/"))):
    print(index,i)

0 business_enterpreteur.csv
1 complaints_corner.csv
2 concerts.csv
3 daily_life.csv
4 employment.csv
5 entertainment_dining.csv
6 family_matters_health.csv
7 finance_banking_taxation.csv
8 food_drink.csv
9 housing.csv
10 insurance.csv
11 introductions.csv
12 language_corner.csv
13 leaving_switzerland.csv
14 market_place.csv
15 off_topic.csv
16 other_general.csv
17 permits_visas_gov.csv
18 pet_corner.csv
19 sports_wellness.csv
20 support.csv
21 swiss_politics_news.csv
22 transportation_driving.csv
23 travel_free_time.csv
24 tv_internet_telephone.csv


# In case we'd like to update the files, let's declare the variables

In [17]:
business_enterpreteur=None
complaints_corner=None
concerts=None
daily_life=None
employment=None
entertainment_dining=None
family_matters_health=None
finance_banking_taxation=None
food_drink=None
housing=None
insurance=None
introductions=None
language_corner=None
leaving_switzerland=None
market_place=None
off_topic=None
other_general=None
permits_visas_gov=None
pet_corner=None
sports_wellness=None
support=None
swiss_politics_news=None
transportation_driving=None
travel_free_time=None
tv_internet_telephone=None
# NON FETCHED BUT YET TO COME
commercial_events=None
education=None
social_events=None
swiss_news=None

# Let's initialize them

In [36]:
for index,i in enumerate(sorted(dataframes)):
    print(index, dataframes[i])
    if index==0:  
        business_enterpreteur=createDF(dataframes[i])
        saveData(business_enterpreteur)
    elif index==1: 
        commercial_events=createDF(dataframes[i])#####
        saveData(commercial_events)
    elif index==2: 
        complaints_corner=createDF(dataframes[i])   
        saveData(complaints_corner)
    elif index==3: 
        concerts=createDF(dataframes[i])
        saveData(concerts)        
    elif index==4: 
        daily_life=createDF(dataframes[i])
        saveData(daily_life)
    elif index==5: 
        education=createDF(dataframes[i])#####
        saveData(education)
    elif index==6: 
        employment=createDF(dataframes[i])
        saveData(employment)
    elif index==7: 
        entertainment_dining=createDF(dataframes[i])
        saveData(entertainment_dining)
    elif index==8: 
        family_matters_health=createDF(dataframes[i])
        saveData(family_matters_health)
    elif index==9: 
        finance_banking_taxation=createDF(dataframes[i])
        saveData(finance_banking_taxation)
    elif index==10: 
        food_drink=createDF(dataframes[i])
        saveData(food_drink)
    elif index==11: 
        housing=createDF(dataframes[i])
        saveData(housing)
    elif index==12: 
        insurance=createDF(dataframes[i])
        saveData(insurance)
    elif index==13: 
        introductions=createDF(dataframes[i])
        saveData(introductions)
    elif index==14: 
        language_corner=createDF(dataframes[i])
        saveData(language_corner)
    elif index==15: 
        leaving_switzerland=createDF(dataframes[i])
        saveData(leaving_switzerland)
    elif index==16: 
        market_place=createDF(dataframes[i])
        saveData(market_place)
    elif index==17: 
        off_topic=createDF(dataframes[i])
        saveData(off_topic)
    elif index==18: 
        other_general=createDF(dataframes[i])
        saveData(other_general)
    elif index==19: 
        permits_visas_gov=createDF(dataframes[i])
        saveData(permits_visas_gov)
    elif index==20: 
        pet_corner=createDF(dataframes[i])
        saveData(pet_corner)
    elif index==21: 
        social_events=createDF(dataframes[i])#####
        saveData(social_events)
    elif index==22: 
        sports_wellness=createDF(dataframes[i])
        saveData(sports_wellness)
    elif index==23: 
        support=createDF(dataframes[i])
        saveData(support)
    elif index==24: 
        swiss_news=createDF(dataframes[i])######
        saveData(swiss_news)
    elif index==25: 
        swiss_politics_news=createDF(dataframes[i])
        saveData(swiss_politics_news)
    elif index==26: 
        tv_internet_telephone=createDF(dataframes[i])
        saveData(tv_internet_telephone)
    elif index==27: 
        transportation_driving=createDF(dataframes[i])
        saveData(transportation_driving)
    else: 
        travel_free_time=createDF(dataframes[i])
        saveData(travel_free_time)

0 http://www.englishforum.ch/business-entrepreneur/
1 http://www.englishforum.ch/commercial-events/
2 http://www.englishforum.ch/complaints-corner/
3 http://www.englishforum.ch/concerts/
4 http://www.englishforum.ch/daily-life/
5 http://www.englishforum.ch/education/
6 http://www.englishforum.ch/employment/
7 http://www.englishforum.ch/entertainment-dining/
8 http://www.englishforum.ch/family-matters-health/
9 http://www.englishforum.ch/finance-banking-taxation/
10 http://www.englishforum.ch/food-drink/
11 http://www.englishforum.ch/housing-general/
12 http://www.englishforum.ch/insurance/
13 http://www.englishforum.ch/introductions/
14 http://www.englishforum.ch/language-corner/
15 http://www.englishforum.ch/leaving-switzerland/
16 http://www.englishforum.ch/market-place/
17 http://www.englishforum.ch/off-topic/
18 http://www.englishforum.ch/other-general/
19 http://www.englishforum.ch/permits-visas-government/
20 http://www.englishforum.ch/pet-corner/
21 http://www.englishforum.ch/so

# Load all data at once

In [40]:
import glob
dfContainer={}#sorted(dataframes)
filenames = glob.glob("csv/*.csv")
for index,filename in enumerate(filenames):
    df = pd.read_csv(filename, header=0,sep='\t',index_col=0)
    dfContainer[re.sub(r'.csv','',os.path.basename(filename))]=df

# sorted(dataframes)
# dataFrames = pd.Series(dataFrames)
# df=pd.read_csv('Concerts',sep='\t',index_col=0)
# df

# dfContainer

## Ideas
- Buscar comentarios más positivos y encontrar los threads de más interes

- con los threads más visitados podemos obtener los topics de más interes
- los usuarios que más aportan en el foro
- los temas más polemicos (tal vez usando los comentarios negativos)
- En un apartado de ventas que hay, intentar sacar lo que se vende en suiza y en donde (aunque el donde no lo tengo muy claro)
- las horas de más publicaciones
- lodgement: tal vez podemos aplicar ML como un clasificador de oportunidades de encontrar casa en suiza

In [525]:
def post(thread):
    # Scrape the HTML at the url
    r = requests.get(url)    
    # Turn the HTML into a Beautiful Soup object
    soup = BeautifulSoup(r.text, 'lxml')
    print(r.text)
    comments={}
#     print(soup.text)
#     for link in soup.get_text().find_all("td",{"class":"alt1"}):
#         for link2 in link.find_all("table"):
#         print(link.parent)
#         for link2 in link.find_all("div",{"id":"posts"})
#             print(link2)
#     for link in soup.find_all("div",{"align":"center"}):
#         for link2 in link.find_all("div",{"align":"left"}):
#             for link3 in link2.find_all("tr"):
#                 for link4 in link3.find_all("td",{"width":"99%"}):
#                     for link5 in link4.find_all("div"):
#                         print(link5)
#                     for link4 in link3.find_all("tbody"):
#                         for link5 in link4.find_all("tr"):
#                             print(link5.contents)
#         print('tableeee',link.attrs,'posts' in link.contents)
#         for link1 in link.find_all("div"):
#             print('diveee','posts' in link1.contents)
#             if('id' in link1.attrs and link1['id']=='posts'):
#         ,id="posts"):
#                 print(link)
        
post('http://www.englishforum.ch/other-general/168845-gun-community.html')

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"> <html xmlns="http://www.w3.org/1999/xhtml" dir="ltr" lang="en"> <head>  <meta http-equiv="Pragma" content="no-cache" /> <meta http-equiv="Expires" content="-1" /> <meta http-equiv="Cache-Control" content="no-cache" />  <meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1" /> <meta name="generator" content="vBulletin 3.8.4" /> <meta name="keywords" content="English information Switzerland Zurich expat activities community forum chat meet foreigner" /> <meta name="description" content="Community portal for English-speaking people in Switzerland" />  <style type="text/css" id="vbulletin_css">
/**
* vBulletin 3.8.4 CSS
* Style: 'Default Style'; Style ID: 1
*/
body
{
	background: #ffffff;
	color: #000000;
	font: 10pt verdana, geneva, lucida, 'lucida grande', arial, helvetica, sans-serif;
	margin: 5px 10px 10px 10px;
	padding: 0px 25px;
}
a:link, body_al